In [3]:
#install gradio
!pip install gradio
!pip install langchain_openai
!pip install gradio_client
!pip install openai

Make simple gradio UI, get more advanced below

In [4]:
#imports
import openai
import gradio as gr
from typing import List, Tuple

#function to be wrapped by gradio UI
def essay_grading(name):
    return "Guten Tag, " + name + "!"

#demo to be shown -- the actual UI part
demo = gr.Interface(
    fn=essay_grading,  #fn is the function being wrapped by the UI
    inputs=["text"],
    outputs=["text"],
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6328b1703bf1509293.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


More complex UI part, AI not implemented

In [7]:
#imports
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

#function to be wrapped by gradio UI
def essay_grading(name):
    return "Guten Tag, " + name + "!"
#test with multiple functions
def essay_grading_negative(name):
    return "NEIN, " + name + "!"


#demo to be shown -- the actual UI part
with gr.Blocks() as demo:  #use gr.Blocks to create more structured layout

  gr.Markdown("# Essay Grader\nver.1.1") # #in markdown means header
  question_textbox = gr.Textbox(label="Input", interactive = True, value = "") #makes textbox at the top of the interface --
  # order in code is order in interface

  with gr.Row(): #puts row-wise layout below what's already made above ^
    #gr.Markdown("# Choose an option; \n slider, textbox, dropdown menu, label, etc. can all be implemented")
    sent_button = gr.Button(value="Send")


  with gr.Column(): #puts column-wise layout below the row (as it's above this ^)
    #gr.Markdown("# Choose an option; \n slider, textbox, dropdown menu, label, etc. can all be implemented\n")
    output_textbox = gr.Textbox(label="Output", interactive=False)
    output_textbox2 = gr.Textbox(label="Output but Bad", interactive=False)

  sent_button.click(
    essay_grading,  #fn is the function being wrapped by the UI
    inputs=[question_textbox],
    outputs=[output_textbox]
   )
  sent_button.click(
    essay_grading_negative,  #fn is the function being wrapped by the UI
    inputs=[question_textbox],
    outputs=[output_textbox2]
   )

demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://de3d68a589417c6f1d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://de3d68a589417c6f1d.gradio.live


implement using chatbot -- SIMPLE

In [ ]:
#imports
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

#get OpenAI API key
from google.colab import userdata
openai.api_key = userdata.get('conner').strip()

#function to be wrapped by gradio UI
def essay_grading(prompt: str, question: str, temp = 1.0) -> str:
    essay_prompt = "Give constructive feedback on this essay: " #prompt for the chatbot to follow
    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "system", "content": essay_prompt}, #dictionaries to define chatbot behavior
              {"role": "user", "content": question} # 3 main roles; system, user, assistant
        ],
            temperature = temp,
            max_tokens=200,
    )
    query_reply = response.choices[0].message.content  #generate response based on first choice given, give the text

    return query_reply #return the chatbot reply




with gr.Blocks() as demo:
  gr.Markdown("# Essay Grader\nver.2.0")

  chatbot = gr.Chatbot() #initialize chatbot
  essay_prompt = "Give constructive feedback on this essay: " #prompt for the chatbot to follow
  prompt_textbox = gr.Textbox(label="Prompt", value=essay_prompt, visible=False) #invisble textbox for the prompt to sit in


  question_textbox = gr.Textbox(label="Input", interactive = True, value = "")
  with gr.Row():
    sent_button = gr.Button(value="Send")
    reset_button = gr.Button(value="Reset")
  with gr.Column():
    output_textbox = gr.Textbox(label="Output", interactive=False)

  sent_button.click(
    essay_grading,
    inputs=[prompt_textbox, question_textbox],
    outputs=[output_textbox]
   )


demo.launch(debug = True)

<ipython-input-11-e683b386c518>:33: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot() #initialize chatbot


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5075ccc14daa4b021d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'

display output through chatbot; remember conversation and reset button working; basically hw2

In [ ]:
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

from google.colab import userdata
openai.api_key = userdata.get('conner').strip()
def essay_grading(prompt: str, question: str, temp = 1.0) -> List[Tuple[str, str]]:  #changed output type to List[Tuple[str, str]] format for
                                                                                     #chatbot preferred output format
    global conversation_history #global variable to hold previous conversations
    conversation_history = [] #initialize as empty list

    conversation_history.append({'role': 'user', 'content': question}) #add previous conversation to chatbot context

#When providing feedback, highlight the word/sentence(s) where the feedback originated
    essay_prompt = "Give constructive feedback on this essay. the format for this feedback is: 'quote where the feedback came from', 'simple feedback' (e.g. 'looks good' is not descriptive). Include only three of the most important feedbacks"
    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": prompt}, #reload the prompt to keep the bot aware of it's prompt
            *conversation_history], #changed messages to conversation_history to reload previous conversations in chatbot response
            temperature = temp,
            max_tokens=200,
    )
    query_reply = response.choices[0].message.content
    conversation_history.append({'role': 'assistant', 'content': query_reply})

    conversation_output = [] #empty conversation_output variable to put the output in and display

    for i in range(0, len(conversation_history)-1, 2):   #scrolls thru and seperates user message and assistant replies from conversation history
        user_message = conversation_history[i]['content']
        assistant_reply = conversation_history[i+1]['content']
        conversation_output.append((user_message, assistant_reply))

    return conversation_output, ""

css_styling = """
h1 {
    font-family: 'Helvetica', sans-serif;
    color: #4CAF50;
    text-align: center;
}
h2 {
    font-family: 'Helvetica', sans-serif;
    color: #333;
}
.highlighted-text {
    background-color: ##a38a65;
    padding: 10px;
    border-radius: 5px;
}
"""

with gr.Blocks(css=css_styling) as demo:
  gr.Markdown("# 📝 Essay Grader\nver.2.0")

  chatbot = gr.Chatbot()
  essay_prompt = "Give constructive feedback on this essay. the format for this feedback is: 'quote where the feedback came from', 'simple feedback' (e.g. 'looks good' is not descriptive). Include only three of the most important feedbacks"
  prompt_textbox = gr.Textbox(label="Prompt", value=essay_prompt, visible=False)


  question_textbox = gr.Textbox(label="Input Essay Contents Here: ", interactive=True, value="", lines=10)
  with gr.Row():
     sent_button = gr.Button(value="📤 Submit Essay", variant="primary")
     reset_button = gr.Button(value="🔄 Reset", variant="secondary")

  sent_button.click(
    essay_grading,
    inputs=[prompt_textbox, question_textbox],
    outputs=[chatbot, question_textbox]  #change output to chatbot in order to display it in the chatbot window
   )

  reset_button.click(
    lambda: ([], ""),  # clear conversation history
    inputs=[],  # clear inputs
    outputs=[chatbot, question_textbox]  # reset chatbot
  )

demo.launch(debug = True)

<ipython-input-12-aba586151ca4>:56: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://300fc1aa165df41825.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'

# when giving feedback, try to have it refer to where in the text the feedback is coming from.# (paragraph #, sentence, etc). Figure out how to highlight certain words.

In [ ]:
import openai
import gradio as gr
from typing import List, Tuple, Dict
from google.colab import userdata

openai.api_key = userdata.get('conner').strip()

def essay_grading(prompt: str, question: str, temp=1.0) -> Tuple[List[Dict[str, str]], Dict[str, List[Dict[str, int | str]]]]:
    global conversation_history
    conversation_history = []

    conversation_history.append({'role': 'user', 'content': question})

    essay_prompt = """Give constructive feedback on this essay. The format for this feedback is: 'quote where the feedback came from', 'simple feedback' (e.g., 'looks good' is not descriptive). Include only three of the most important feedbacks."""
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt}, *conversation_history],
        temperature=temp,
        max_tokens=200,
    )
    query_reply = response.choices[0].message.content
    conversation_history.append({'role': 'assistant', 'content': query_reply})

    # Parse the feedback to extract highlighted sections
    highlighted_text = {
        "text": question,  # Include the full essay text
        "entities": []  # List to store highlighted sections
    }

    # Split the feedback into lines and look for the quoted text
    for line in query_reply.split("\n"):
        if "'" in line:  # Look for lines with quotes
            quoted_text = line.split("'")[1]  # Extract the quoted text
            start_idx = question.find(quoted_text)
            if start_idx != -1:  # Ensure the quoted text is found in the essay
                end_idx = start_idx + len(quoted_text)
                highlighted_text["entities"].append({"start": start_idx, "end": end_idx, "label": "critique"})

    conversation_output = []
    for i in range(0, len(conversation_history) - 1, 2):
        user_message = conversation_history[i]['content']
        assistant_reply = conversation_history[i + 1]['content']
        conversation_output.append({"role": "user", "content": user_message})
        conversation_output.append({"role": "assistant", "content": assistant_reply})

    return conversation_output, highlighted_text

# Custom CSS for styling
custom_css = """
h1 {
    font-family: 'Helvetica', sans-serif;
    color: #4CAF50;
    text-align: center;
}
h2 {
    font-family: 'Helvetica', sans-serif;
    color: #333;
}
.highlighted-text {
    background-color: #f0f8ff;
    padding: 10px;
    border-radius: 5px;
}
"""

with gr.Blocks(css=custom_css) as demo:
    # Header
    gr.Markdown("# 📝 Essay Grader\n### Your AI-powered essay feedback tool")

    # Description
    gr.Markdown("""
    Welcome to the Essay Grader! Input your essay below, and the AI will provide constructive feedback.
    The feedback will highlight specific parts of your essay and suggest improvements.
    """)

    # Chatbot and Highlighted Text
    with gr.Row():
        chatbot = gr.Chatbot(label="AI Feedback", height=400, type="messages")  # Use type="messages"
        highlighted_text = gr.HighlightedText(label="Highlighted Essay", show_legend=True)  # Remove height parameter

    # Input Essay
    question_textbox = gr.Textbox(label="Input Essay Contents Here: ", interactive=True, value="", lines=10, placeholder="Paste your essay here...")

    # Controls
    with gr.Row():
        sent_button = gr.Button(value="📤 Submit Essay", variant="primary")
        reset_button = gr.Button(value="🔄 Reset", variant="secondary")

    # Temperature Slider
    temp_slider = gr.Slider(minimum=0.1, maximum=1.0, value=0.7, label="Creativity Level (Temperature)", step=0.1)

    # Hidden Prompt
    essay_prompt = """Give constructive feedback on this essay. The format for this feedback is: 'quote where the feedback came from', 'simple feedback' (e.g., 'looks good' is not descriptive). Include only three of the most important feedbacks."""
    prompt_textbox = gr.Textbox(label="Prompt", value=essay_prompt, visible=False)

    # Button Actions
    sent_button.click(
        essay_grading,
        inputs=[prompt_textbox, question_textbox, temp_slider],
        outputs=[chatbot, highlighted_text]
    )

    reset_button.click(
        lambda: ([], {"text": "", "entities": []}),  # Clear chatbot and highlighted text
        inputs=[],
        outputs=[chatbot, highlighted_text]
    )

# Launch the app
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://559128b829dd7b8a0d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'

In [42]:
def generate_feedback(input_text):
  inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
  output = model.generate(**inputs, max_new_tokens=100)
  return tokenizer.decode(output[0])


css_styling = """
h1 {
    font-family: 'Helvetica', sans-serif;
    color: #4CAF50;
    text-align: center;
}
.gradio-container {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
}
.gradio-textbox {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
    font-size: 16px;
}
.gradio-button {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
    font-size: 16px;
    background-color: #4CAF50;
    color: white;
}
.gradio-button:hover {
    background-color: #45a049;
    color: orange;
}
.gradio-button:active {
    background-color: #3e8e3c;
    color: white;
}
.hidden {
    display: none;
}
.expanded {
    height: 300px !important;
}
"""

def show_true():
    return gr.Textbox.update(visible=True)
def show_false():
    return gr.Textbox.update(visible=False)

def reset_interface():
    return (
        gr.Textbox.update(value="", lines=10, visible=True),
        gr.Textbox.update(value="", visible=True),
        gr.Textbox.update(value="", visible=True),
        gr.Textbox.update(value="", visible=True),
        gr.Textbox.update(value="", visible=False),
        gr.Textbox.update(value="", visible=False),
        gr.Textbox.update(value="", visible=False),
        gr.Textbox.update(value="", visible=False)
    )

def submit_button_updates(essay, requirements, student_id, assignment_id):
    feedback = generate_feedback(essay)
    return (
        gr.Textbox.update(visible=False),
        gr.Textbox.update(visible=False),
        gr.Textbox.update(visible=False),
        gr.Textbox.update(visible=False),
        gr.Textbox.update(value=feedback, visible=True, lines=10),
        gr.Textbox.update(visible=True),
        gr.Textbox.update(visible=True),
        gr.Textbox.update(visible=True)
    )
with gr.Blocks(css=css_styling) as demo:
  gr.Markdown("# 📝 Tropos Essay Grader")
  question_textbox = gr.Textbox(label="Input Essay Contents Here: ", interactive=True, value="", lines=10)
  with gr.Row():
     requirements_input = gr.Textbox(label="Rubric / Requirements: ", interactive=True, value="", lines=3)
  with gr.Row():
     student_id_textbox = gr.Textbox(label="Student ID: ", interactive=True, value="", lines=1)
     assignment_id_textbox = gr.Textbox(label="Assignment ID: ", interactive=True, value="", lines=1)
  with gr.Row():
     sent_button = gr.Button(value="📤 Submit Essay", variant="primary")
     reset_button = gr.Button(value="🔄 Reset", variant="secondary")
  with gr.Column():
    feedback_textbox = gr.Textbox(label="Essay Feedback", interactive=False, value="", lines=5, visible = False)
  with gr.Row():
    inline_textbox1 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible = False)
    inline_textbox2 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible = False)
    inline_textbox3 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible = False)


  sent_button.click(
    submit_button_updates,
    inputs=[question_textbox, requirements_input, student_id_textbox, assignment_id_textbox],
    outputs=[question_textbox, requirements_input, student_id_textbox, assignment_id_textbox, feedback_textbox, inline_textbox1, inline_textbox2, inline_textbox3],
    )
  reset_button.click(
    reset_interface,  # Clear all inputs and feedback
    inputs=[],  # clear inputs
    outputs=[question_textbox, feedback_textbox, assignment_id_textbox, student_id_textbox, requirements_input, inline_textbox1, inline_textbox2, inline_textbox3])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://380a5490ef464f03f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


MAKE A INPUT CLASS AND STUFF -- USING EXAMPLE JOSH (@Oxulotl)
 GAVE IN DISCORD

In [ ]:
import gradio as gr
import json

# Define InputFields and OutputFields classes
class InputFields:
    def __init__(self):
        self._student_id = None
        self._assignment_id = None
        self._requirements_input = None
        self._student_essay = None

    # getters
    def get_student_id(self):
        return self._student_id

    def get_assignment_id(self):
        return self._assignment_id

    def get_requirements_input(self):
        return self._requirements_input

    def get_student_essay(self):
        return self._student_essay

    #setters
    def set_student_id(self, value):
        self._student_id = value

    def set_assignment_id(self, value):
        self._assignment_id = value

    def set_requirements_input(self, value):
        self._requirements_input = value

    def set_student_essay(self, value):
        self._student_essay = value

    #adder methods (builder method)
    def add_student_id(self, value):
        self._student_id = value
        return self

    def add_assignment_id(self, value):
        self._assignment_id = value
        return self

    def add_requirements_input(self, value):
        self._requirements_input = value
        return self

    def add_student_essay(self, value):
        self._student_essay = value
        return self

    # JSON parsing and serialization
    @staticmethod
    def from_json(json_str):
        data = json.loads(json_str)
        obj = InputFields()
        obj.set_student_id(data.get("StudentID"))
        obj.set_assignment_id(data.get("AssignmentID"))
        obj.set_requirements_input(data.get("Requirements Input"))
        obj.set_student_essay(data.get("Student Essay"))
        return obj

    def to_json(self):
        return json.dumps({
            "StudentID": self._student_id,
            "AssignmentID": self._assignment_id,
            "Requirements Input": self._requirements_input,
            "Student Essay": self._student_essay
        }, indent=4)

class OutputFields:
    def __init__(self):
        self._student_id = None
        self._assignment_id = None
        self._text_snippet = None
        self._feedback = None
        self._feedback_type = None

    # Getters
    def get_student_id(self):
        return self._student_id

    def get_assignment_id(self):
        return self._assignment_id

    def get_text_snippet(self):
        return self._text_snippet

    def get_feedback(self):
        return self._feedback

    def get_feedback_type(self):
        return self._feedback_type

    # Setters
    def set_student_id(self, value):
        self._student_id = value

    def set_assignment_id(self, value):
        self._assignment_id = value

    def set_text_snippet(self, value):
        self._text_snippet = value

    def set_feedback(self, value):
        self._feedback = value

    def set_feedback_type(self, value):
        self._feedback_type = value

    # Builder methods (adders)
    def add_student_id(self, value):
        self._student_id = value
        return self

    def add_assignment_id(self, value):
        self._assignment_id = value
        return self

    def add_text_snippet(self, value):
        self._text_snippet = value
        return self

    def add_feedback(self, value):
        self._feedback = value
        return self

    def add_feedback_type(self, value):
        self._feedback_type = value
        return self

    # JSON parsing and serialization
    @staticmethod
    def from_json(json_str):
        data = json.loads(json_str)
        obj = OutputFields()
        obj.set_student_id(data.get("Student ID"))
        obj.set_assignment_id(data.get("Assignment ID"))
        obj.set_text_snippet(data.get("Text Snippet"))
        obj.set_feedback(data.get("Feedback"))
        obj.set_feedback_type(data.get("FeedbackType"))
        return obj

    def to_json(self):
        return json.dumps({
            "Student ID": self._student_id,
            "Assignment ID": self._assignment_id,
            "Text Snippet": self._text_snippet,
            "Feedback": self._feedback,
            "FeedbackType": self._feedback_type
        })


# Dummy feedback
def generate_feedback(input_text):
    #filler code
    return f"Feedback for: {input_text}"


# Custom CSS for styling
css_styling = """
h1 {
    font-family: 'Helvetica', sans-serif;
    color: #4CAF50;
    text-align: center;
}
.gradio-container {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
}
.gradio-textbox {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
    font-size: 16px;
}
.gradio-button {
    font-family: 'Helvetica', sans-serif;
    text-align: center;
    font-size: 16px;
    background-color: #4CAF50;
    color: white;
}
.gradio-button:hover {
    background-color: #45a049;
    color: orange;
}
.gradio-button:active {
    background-color: #3e8e3c;
    color: white;
}
.hidden {
    display: none;
}
.expanded {
    height: 300px !important;
}
"""


def reset_interface():
    return (
        gr.update(value="", visible=True),  #essay textbox
        gr.update(value="", visible=True),  # requirements
        gr.update(value="", visible=True),  # student ID
        gr.update(value="", visible=True),  # assignment ID
        gr.update(value="", visible=False),  #feedback
        gr.update(value="", visible=False),  # inline feedback 1
        gr.update(value="", visible=False),  # inline feedback 2
        gr.update(value="", visible=False)   #inline feedback 3
    )


def submit_button_updates(essay, requirements, student_id, assignment_id):
    input_data = InputFields() \
        .add_student_id(student_id) \
        .add_assignment_id(assignment_id) \
        .add_requirements_input(requirements) \
        .add_student_essay(essay)

    feedback = generate_feedback(input_data.get_student_essay())#feedback

    output_data = OutputFields() \
        .add_student_id(input_data.get_student_id()) \
        .add_assignment_id(input_data.get_assignment_id()) \
        .add_text_snippet(input_data.get_student_essay()) \
        .add_feedback(feedback) \
        .add_feedback_type("Constructive")  #feedback type

    return (
        gr.update(visible=False),  #essay input
        gr.update(visible=False),  #rubric/requirements
        gr.update(visible=False),  #student ID
        gr.update(visible=False),  #assignment ID
        gr.update(value=output_data.get_feedback(), visible=True, lines=10),  #feedback
        gr.update(visible=True),  #inline feedback 1
        gr.update(visible=True),  #inline feedback 2
        gr.update(visible=True)   #inline feedback 3
    )


with gr.Blocks(css=css_styling) as demo:

    gr.Markdown("# 📝 Tropos Essay Grader")
    question_textbox = gr.Textbox(label="Input Essay Contents Here: ", interactive=True, value="", lines=10)
    requirements_input = gr.Textbox(label="Rubric / Requirements: ", interactive=True, value="", lines=3)

    with gr.Row():
        student_id_textbox = gr.Textbox(label="Student ID: ", interactive=True, value="", lines=1)
        assignment_id_textbox = gr.Textbox(label="Assignment ID: ", interactive=True, value="", lines=1)

    with gr.Row():
        sent_button = gr.Button(value="📤 Submit Essay", variant="primary")
        reset_button = gr.Button(value="🔄 Reset", variant="secondary")

    feedback_textbox = gr.Textbox(label="Essay Feedback", interactive=False, value="", lines=5, visible=False)

    with gr.Row():
        inline_textbox1 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible=False)
        inline_textbox2 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible=False)
        inline_textbox3 = gr.Textbox(label="Inline Feedback", interactive=False, value="", lines=3, visible=False)

    sent_button.click(
        submit_button_updates,
        inputs=[question_textbox, requirements_input, student_id_textbox, assignment_id_textbox],
        outputs=[question_textbox, requirements_input, student_id_textbox, assignment_id_textbox, feedback_textbox, inline_textbox1, inline_textbox2, inline_textbox3],
    )

    reset_button.click(
        reset_interface,  
        inputs=[],  # clear inputs
        outputs=[question_textbox, requirements_input, student_id_textbox, assignment_id_textbox, feedback_textbox, inline_textbox1, inline_textbox2, inline_textbox3]
    )

# Launch the interface
demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1f1be7b85561dcad02.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7898 <> https://1f1be7b85561dcad02.gradio.live
